# Prerequisites

In [3]:
%pip install scikit-learn
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ------- -------------------------------- 2.1/11.6 MB 11.8 MB/s eta 0:00:01
   --------------- ------------------------ 4.5/11.6 MB 11.7 MB/s eta 0:00:01
   ------------------------ --------------- 7.1/11.6 MB 11.8 MB/s eta 0:00:01
   -------------------------------- ------- 9.4/11.6 MB 11.7 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 11.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


# 1. Download Dataset

In [2]:
filename = "used_cars_autoscout24.csv"
URL = "http://www.juergenbrauer.org/datasets/" + filename

# import urllib.request
# urllib.request.urlretrieve(URL, filename)

# 2. Read Data

In [3]:
import pandas
table = pandas.read_csv(filename)
table.head(5)

,Unnamed: 0,brand,model,color,registration_date,year,price_in_euro,power_kw,power_ps,transmission_type,fuel_type,fuel_consumption_l_100km,fuel_consumption_g_km,mileage_in_km,offer_description
0,0,alfa-romeo,Alfa Romeo GTV,red,10/1995,1995,1300,148,201,Manual,Petrol,"10,9 l/100 km",260 g/km,160500.0,2.0 V6 TB
1,1,alfa-romeo,Alfa Romeo 164,black,02/1995,1995,24900,191,260,Manual,Petrol,NaN,- (g/km),190000.0,"Q4 Allrad, 3.2L GTA"
2,2,alfa-romeo,Alfa Romeo Spider,black,02/1995,1995,5900,110,150,Unknown,Petrol,NaN,- (g/km),129000.0,ALFA ROME 916
3,3,alfa-romeo,Alfa Romeo Spider,black,07/1995,1995,4900,110,150,Manual,Petrol,"9,5 l/100 km",225 g/km,189500.0,2.0 16V Twin Spark L
4,4,alfa-romeo,Alfa Romeo 164,red,11/1996,1996,17950,132,179,Manual,Petrol,"7,2 l/100 km",- (g/km),96127.0,"3.0i Super V6, absoluter Topzustand !"


# 3. Clean Data

In [13]:
table["registration_date"] = pandas.to_datetime(table["registration_date"], errors="coerce")
table["year"] = pandas.to_numeric(table["year"], errors="coerce")
table["price_in_euro"] = pandas.to_numeric(table["price_in_euro"], errors="coerce")
table["power_kw"] = pandas.to_numeric(table["power_kw"], errors="coerce")
table["power_ps"] = pandas.to_numeric(table["power_ps"], errors="coerce")
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251079 entries, 0 to 251078
Data columns (total 15 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   Unnamed: 0                251079 non-null  int64         
 1   brand                     251079 non-null  object        
 2   model                     251079 non-null  object        
 3   color                     250913 non-null  object        
 4   registration_date         250882 non-null  datetime64[ns]
 5   year                      250884 non-null  float64       
 6   price_in_euro             250880 non-null  float64       
 7   power_kw                  250786 non-null  float64       
 8   power_ps                  250869 non-null  float64       
 9   transmission_type         251079 non-null  object        
 10  fuel_type                 251079 non-null  object        
 11  fuel_consumption_l_100km  224206 non-null  object        
 12  fu

In [14]:
clean_data = table[["brand", "mileage_in_km", "power_ps", "price_in_euro"]]
clean_data = clean_data.dropna()
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250692 entries, 0 to 251078
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   brand          250692 non-null  object 
 1   mileage_in_km  250692 non-null  float64
 2   power_ps       250692 non-null  float64
 3   price_in_euro  250692 non-null  float64
dtypes: float64(3), object(1)
memory usage: 9.6+ MB


In [15]:
# One Hot Encoding
clean_data = pandas.get_dummies(clean_data)

# 4. Prepare Data for ML

##  4.1 Input/Output Split

In [58]:
x = clean_data.drop("price_in_euro", axis="columns")
y = clean_data[["price_in_euro"]]

In [29]:
x.shape

(250692, 49)

In [31]:
y.shape

(250692, 1)

## 4.2 Train/Test Split

In [70]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [72]:
x_train.shape

(200553, 49)

In [74]:
x_test.shape

(50139, 49)

In [76]:
y_train.shape

(200553, 1)

In [78]:
y_test.shape

(50139, 1)

## 4.3 Scale


In [87]:
from sklearn.preprocessing import MinMaxScaler

input_scaler = MinMaxScaler()
input_scaler.fit(x_train)
x_train_scaled = input_scaler.transform(x_train)

output_scaler = MinMaxScaler()
output_scaler.fit(y_train)
y_train_scaled = output_scaler.transform(y_train)